In [4]:
# -*- coding: utf-8 -*-
# Océane /skincare — API (URLs, preço, imagem fallback) + PDP HTML (ingredientes, benefícios, imagem) — 1 célula

import os, re, json, time, random, unicodedata
from urllib.parse import urlsplit
from pathlib import Path
from typing import List, Dict, Optional

import requests
import pandas as pd
from bs4 import BeautifulSoup

# ========================= CONFIG =========================
BASE_URL = "https://www.oceane.com.br"
CAT_PATH = "skincare"
BRAND = "oceane"

BATCH = 50
REQ_TIMEOUT = 25
RETRY = 3
SLEEP = (0.5, 1.0)

SESSION = requests.Session()
HEADERS = {
    "User-Agent": ("Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
                   "(KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"),
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,application/json;q=0.8,*/*;q=0.7",
    "Accept-Language": "pt-BR,pt;q=0.9,en-US;q=0.6,en;q=0.5",
    "Origin": BASE_URL,
    "Referer": f"{BASE_URL}/{CAT_PATH}",
}

IMAGES_DIR = Path("images")
IMAGES_DIR.mkdir(parents=True, exist_ok=True)
DOWNLOAD_IMAGES = True

# ========================= SEUS MÓDULOS =========================
import sys
sys.path.append(os.path.abspath("/home/usuario/Área de trabalho/Dados/models"))

try:
    from skin import SKIN_TYPE_CANONICAL_ORDER, SKIN_TYPE_SYNONYMS_PT
except Exception:
    SKIN_TYPE_CANONICAL_ORDER = ["seca","oleosa","mista","normal","sensível","acneica","madura","todas"]
    SKIN_TYPE_SYNONYMS_PT = {
        "seca":["pele seca","secas"],
        "oleosa":["oleosa","oleosas","oleosidade","sebo","seborreguladora"],
        "mista":["mista","mistas"],
        "normal":["normal","normais"],
        "sensível":["sensível","sensíveis","sensibilidade"],
        "acneica":["acne","espinhas","acneica","acneicas"],
        "madura":["madura","rugas","linhas finas","anti-idade"],
        "todas":["todos os tipos","todas as peles","todas"]
    }

try:
    from exclude import EXCLUDE_KEYWORDS
except Exception:
    EXCLUDE_KEYWORDS = []

try:
    from ingredient import INGREDIENTES_VALIDOS as _IV
    INGREDIENTES_VALIDOS = set(s.strip().casefold() for s in _IV)
except Exception:
    INGREDIENTES_VALIDOS = {
        "retinol","niacinamida","vitamina c","ácido hialurônico","hialuronato de sódio",
        "cafeína","taurina","pantenol","glicerina","tocoferol","ceramidas",
        "ácido salicílico","centella asiatica","aloe vera","madecassoside","ceramide np"
    }

try:
    from benefits import BENEFIT_SYNONYMS_PT, BENEFIT_CANONICAL_ORDER
except Exception:
    BENEFIT_SYNONYMS_PT = {
        "hidratação": ["hidrata","hidratada","hidratante","hidratação"],
        "acalma": ["acalma","calmante","suaviza","ameniza vermelhidão","reduz vermelhidão"],
        "antioxidante": ["antioxidante","protege contra radicais livres"],
        "antiolheiras": ["olheiras","antiolheiras","clareia olheiras"],
        "diminui inchaço/bolsas": ["inchaço","bolsas","desincha","drenante"],
        "controle de oleosidade": ["controla a oleosidade","seborreguladora","reduz sebo"],
        "anti-idade": ["rugas","linhas finas","anti-idade","firma","firmeza","colágeno"],
        "uniformiza tom": ["uniformiza","clareador","hiperpigmentação","manchas"],
        "regeneração": ["regeneração","repara","barreira da pele","cicatrizante"],
    }
    BENEFIT_CANONICAL_ORDER = [
        "hidratação","controle de oleosidade","antioxidante","acalma",
        "anti-idade","uniformiza tom","antiolheiras","diminui inchaço/bolsas","regeneração"
    ]

try:
    from category import CATEGORY_CANONICAL_ORDER, CATEGORY_HINTS
except Exception:
    CATEGORY_CANONICAL_ORDER = ["limpeza","tônico","esfoliante","máscara","hidratante","sérum","protetor solar","tratamento para área dos olhos","lábios","acessórios","kits","outros"]
    CATEGORY_HINTS = {
        "tratamento para área dos olhos":["olhos","eye","olheira","olheiras","bolsas","eye cream","cica eye"],
        "sérum":["serum","sérum"],
        "hidratante":["creme","gel-creme","loção","hidratante","gel hidratante","cream"],
        "limpeza":["sabonete","gel de limpeza","cleanser","demaquilante","cleansing","balm demaquilante"],
        "máscara":["máscara","mask"],
        "protetor solar":["fps","protetor","sunscreen","solar"],
        "lábios":["lip","lábios","balm","lip balm"],
        "kits":["kit","combo","k"]
    }

# ========================= HELPERS =========================
def _sleep(): time.sleep(random.uniform(*SLEEP))
def clean_space(s): return " ".join((s or "").split())
def normcase(s): return unicodedata.normalize("NFKC", (s or "")).strip().casefold()

def strip_accents_lower(s):
    if not s: return ""
    s = "".join(c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c) != "Mn")
    s = s.lower()
    s = re.sub(r"[^\w\s-]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def fetch(url: str, params: dict | None = None):
    for attempt in range(1, RETRY+1):
        try:
            _sleep()
            r = SESSION.get(url, headers=HEADERS, params=params or {}, timeout=REQ_TIMEOUT)
            if 200 <= r.status_code < 300:
                return r
            print(f"[HTTP] {r.status_code} -> {r.url} (tentativa {attempt})")
            if r.status_code in (403, 429): time.sleep(2.0 * attempt)
            else: time.sleep(0.7 * attempt)
        except requests.RequestException as e:
            print(f"[NET] {e.__class__.__name__} -> {url} (tentativa {attempt})")
            time.sleep(1.0 * attempt)
    return None

def get_soup(url: str) -> Optional[BeautifulSoup]:
    r = fetch(url)
    if not r: return None
    return BeautifulSoup(r.text, "lxml")

def only_filename_from_url(u: str) -> str:
    if not u: return ""
    return os.path.basename(urlsplit(u)._replace(query="").path) or ""

def sanitize_filename(name: str) -> str:
    s = strip_accents_lower(name)
    s = re.sub(r"[^a-z0-9]+", "-", s).strip("-")
    s = re.sub(r"-{2,}", "-", s)
    return s or "produto"

def download_image(image_url: str, product_name: str) -> str:
    if not image_url: return ""
    try:
        r = fetch(image_url)
        if not r: return ""
        # extensão
        ext = ".jpg"
        lu = image_url.lower()
        if ".png" in lu: ext = ".png"
        elif ".webp" in lu: ext = ".webp"
        base = sanitize_filename(product_name)
        dest = IMAGES_DIR / f"{base}{ext}"
        c = 1
        while dest.exists():
            dest = IMAGES_DIR / f"{base}-{c}{ext}"
            c += 1
        with open(dest, "wb") as f:
            f.write(r.content)
        return dest.name
    except Exception as e:
        print(f"[IMG] falha {e} -> {image_url}")
        return ""

# ========================= NORMALIZAÇÕES =========================
QTY_RX = re.compile(r"(?P<val>\d+[.,]?\d*)\s*(?P<u>ml|mL|l|g|mg|kg|un|und|unid|gr|grs)\b", re.I)

def extract_quantity(*texts) -> str:
    blob = " ".join([t for t in texts if t])
    m = QTY_RX.search(blob or "")
    if not m: return ""
    val = m.group("val").replace(",", ".")
    u = m.group("u").lower()
    u = {"ml":"ml","mL":"ml","l":"l","g":"g","mg":"mg","kg":"kg","un":"un","und":"un","unid":"un","gr":"g","grs":"g"}.get(u,u)
    try:
        if "." in val and float(val).is_integer(): val = str(int(float(val)))
    except: pass
    return f"{val}{u}"

def normalize_category_from_texts(*texts) -> str:
    blob_ci = normcase(" ".join([t for t in texts if t]))
    for canon, hints in CATEGORY_HINTS.items():
        if any(normcase(h) in blob_ci for h in hints): return canon
    for canon in CATEGORY_CANONICAL_ORDER:
        if normcase(canon) in blob_ci: return canon
    return "outros"

def normalize_skin_types(*texts) -> str:
    blob_ci = normcase(" ".join([t for t in texts if t]))
    found = set()
    for canon, syns in SKIN_TYPE_SYNONYMS_PT.items():
        pool = syns if isinstance(syns, (list,tuple,set)) else [syns]
        if any(normcase(s) in blob_ci for s in pool) or normcase(canon) in blob_ci:
            found.add(canon)
    order = SKIN_TYPE_CANONICAL_ORDER or []
    return "; ".join([c for c in order if c in found]) if order else "; ".join(sorted(found))

def looks_like_ingredients(txt: str) -> bool:
    """Heurística: muito separador, nomes químicos/INCI, vírgulas restantes."""
    if not txt: return False
    t = txt.strip()
    commas = t.count(",")
    # Se tiver muitas vírgulas e vários termos conhecidos, provavelmente é INCI/composição
    hits = sum(1 for k in ["gly", "niacin", "acid", "extract", "ceramide", "hyalur", "alcohol", "parfum", "lecithin", "glycer", "stear"] if k in t.lower())
    return commas >= 8 and hits >= 2

def normalize_ingredients(text: str) -> str:
    if not text: return ""
    # cortar após "Composição"/"Ingredientes" se houver
    cut = re.split(r"(?i)\b(composição|ingredientes)\b[:\-]?\s*", text, maxsplit=1)
    raw = cut[2] if len(cut) >= 3 else text
    parts = re.split(r"[;,/]\s*|\s+\-\s+|\s•\s|•", raw)
    parts = [p.strip() for p in parts if p and p.strip()]
    if INGREDIENTES_VALIDOS:
        filt = []
        for p in parts:
            p_ci = strip_accents_lower(p).replace("acido hialuronico","ácido hialurônico")
            if (p_ci in INGREDIENTES_VALIDOS) or any(p_ci.startswith(k) for k in INGREDIENTES_VALIDOS):
                filt.append(p_ci)
        parts = filt if filt else parts  # se nada bater, mantém lista original
    # dedup mantendo ordem
    seen, out = set(), []
    for p in parts:
        if p not in seen:
            seen.add(p); out.append(p)
    return "; ".join(out)

def extract_benefits_from_text(text: str) -> str:
    """Mapeia texto livre -> benefícios canônicos via BENEFIT_SYNONYMS_PT."""
    if not text: return ""
    n = strip_accents_lower(text)
    found = set()
    for canonico, patt_list in BENEFIT_SYNONYMS_PT.items():
        for patt in patt_list:
            if patt and strip_accents_lower(patt) in n:
                found.add(canonico); break
    if not found: return ""
    if BENEFIT_CANONICAL_ORDER:
        order = {b:i for i,b in enumerate(BENEFIT_CANONICAL_ORDER)}
        return "; ".join(sorted(found, key=lambda x: order.get(x, 999)))
    return "; ".join(sorted(found))

# ========================= VTEX SEARCH API =========================
def api_list_batch(offset: int, size: int) -> list:
    url = f"{BASE_URL}/api/catalog_system/pub/products/search/{CAT_PATH}"
    params = {"map":"c", "_from": offset, "_to": offset + size - 1}
    r = fetch(url, params=params)
    if not r: return []
    try:
        data = r.json()
        return data if isinstance(data, list) else []
    except Exception:
        return []

def api_iter_all() -> list:
    items, off = [], 0
    while True:
        batch = api_list_batch(off, BATCH)
        if not batch: break
        items.extend(batch)
        print(f"[API] +{len(batch)} (total {len(items)})")
        if len(batch) < BATCH: break
        off += BATCH
    return items

# ========================= PDP HTML EXTRACT =========================
def extract_pdp_fields(url: str, api_fallback: dict | None) -> dict:
    """Retorna dict com ingredientes, beneficios, tipo_pele extra, quantidade extra e imagem baixada."""
    out = {"ingredientes": "", "beneficios": "", "tipo_pele_extra": "", "quantidade_extra": "", "imagem_fn": "", "nome_html": "", "categoria_html": ""}
    soup = get_soup(url)
    if not soup: 
        return out

    # Nome (às vezes melhor no HTML)
    name = ""
    for sel in ["h1.productName","h1.vtex-store-components-3-x-productNameContainer",".vtex-store-components-3-x-productBrand","h1"]:
        n = soup.select_one(sel)
        if n:
            name = clean_space(n.get_text(" ", strip=True))
            break
    out["nome_html"] = name

    # Breadcrumb → categoria extra
    bc = soup.select_one('[data-testid="breadcrumb"]') or soup.select_one(".vtex-breadcrumb-1-x-container")
    breadcrumb_text = clean_space(bc.get_text(" ", strip=True)) if bc else ""
    out["categoria_html"] = normalize_category_from_texts(breadcrumb_text, name)

    # Blocos "specifications" / "display:contents"
    content_divs = soup.select(".vtex-store-components-3-x-content--specifications-tabs div[style*='display:contents']")
    texts = [clean_space(d.get_text(" ", strip=True)) for d in content_divs if d]
    texts = [t for t in texts if t]

    # Ingredientes: procurar por "Composição" antes OU heurística química
    ingredientes = ""
    # procura tag que mencione Composição e pega o próximo display:contents
    comp_label = None
    for tag in soup.find_all(text=re.compile(r"(?i)composi[cç][aã]o|ingredientes")):
        comp_label = tag
        break
    if comp_label:
        # pega o próximo div display:contents depois do label
        cont = None
        node = comp_label.parent
        for _ in range(8):
            if not node: break
            node = node.find_next()
            if node and node.name == "div" and node.get("style") and "display:contents" in node.get("style"):
                cont = node; break
        if cont:
            ingredientes = clean_space(cont.get_text(" ", strip=True))

    if not ingredientes:
        # heurística: escolha o primeiro bloco que "parece" INCI
        for t in texts:
            if looks_like_ingredients(t):
                ingredientes = t; break

    # Benefícios: concatenar blocos descritivos (que não parecem INCI)
    benefits_texts = [t for t in texts if not looks_like_ingredients(t)]
    beneficios = extract_benefits_from_text(" ".join(benefits_texts))

    # Tipos de pele extra e quantidade extra vindos da PDP
    tipo_pele_extra = normalize_skin_types(name, " ".join(texts))
    quantidade_extra = extract_quantity(name, " ".join(texts))

    # Imagem principal: pegar maior do srcset
    def best_src_from_img(img):
        if not img: return ""
        if img.get("srcset"):
            best = ""
            best_w = 0
            for part in img["srcset"].split(","):
                part = part.strip()
                if " " in part:
                    u, w = part.rsplit(" ", 1)
                    w = w.strip().lower().replace("w","")
                    try:
                        wi = int(w)
                        if wi > best_w:
                            best_w = wi; best = u.strip()
                    except:
                        continue
            if best: return best
        return img.get("src") or ""
    img_el = soup.select_one("img.vtex-store-components-3-x-productImageTag--main") or \
             soup.select_one("img.vtex-store-components-3-x-productImageTag")
    img_url = best_src_from_img(img_el) if img_el else ""
    img_fn = ""
    if not img_url and api_fallback:
        # usa imagem da API como fallback
        items = api_fallback.get("items") or []
        for it in items:
            imgs = it.get("images") or []
            if imgs:
                img_url = imgs[0].get("imageUrl") or imgs[0].get("imageUrlHttps") or ""
                if img_url: break

    if img_url and DOWNLOAD_IMAGES:
        img_fn = download_image(img_url, name or (api_fallback.get("productName") if api_fallback else "produto"))
    elif img_url:
        img_fn = only_filename_from_url(img_url)

    out.update({
        "ingredientes": normalize_ingredients(ingredientes),
        "beneficios": beneficios,
        "tipo_pele_extra": tipo_pele_extra,
        "quantidade_extra": quantidade_extra,
        "imagem_fn": img_fn
    })
    return out

# ========================= PARSE (API + PDP) =========================
def parse_product(api_obj: dict) -> Optional[Dict]:
    name = clean_space(api_obj.get("productName") or api_obj.get("productTitle") or api_obj.get("productReference") or "")
    if not name: return None
    if any(k.lower() in name.lower() for k in EXCLUDE_KEYWORDS): return None

    # básicos via API
    categories = api_obj.get("categories") or []
    breadcrumb = " / ".join(categories)
    meta_desc = (api_obj.get("metaTagDescription") or "").strip()
    long_desc = (api_obj.get("description") or api_obj.get("productDescription") or meta_desc).strip()

    categoria = normalize_category_from_texts(breadcrumb, name)
    quantidade = extract_quantity(name, long_desc)
    tipo_pele = normalize_skin_types(name, meta_desc, long_desc)

    # preço
    preco = ""
    items = api_obj.get("items") or []
    try:
        for it in items:
            for seller in it.get("sellers", []):
                co = seller.get("commertialOffer") or {}
                price = co.get("Price") or co.get("price")
                if price is not None:
                    preco = f"{float(price):.2f}"
                    break
            if preco: break
    except Exception:
        preco = ""

    # imagem fallback (se PDP falhar)
    img_url_api = ""
    for it in items:
        imgs = it.get("images") or []
        if imgs:
            img_url_api = imgs[0].get("imageUrl") or imgs[0].get("imageUrlHttps") or ""
            if img_url_api: break

    # URL da PDP para enriquecer
    slug = api_obj.get("linkText") or ""
    pdp_url = f"{BASE_URL}/{slug}/p" if slug else ""

    # Enriquecimento pela PDP
    pdp = extract_pdp_fields(pdp_url, api_obj) if pdp_url else {}

    # Campos finais (prioriza PDP quando existir)
    nome_final = pdp.get("nome_html") or name
    categoria_final = pdp.get("categoria_html") or categoria
    quantidade_final = pdp.get("quantidade_extra") or quantidade
    tipo_pele_final = pdp.get("tipo_pele_extra") or tipo_pele
    ingredientes_final = pdp.get("ingredientes") or normalize_ingredients(long_desc)
    beneficios_final = pdp.get("beneficios") or ""  # mapeado por sinônimos

    # imagem: prioriza a baixada da PDP; senão usa API
    imagem_fn = pdp.get("imagem_fn") or (download_image(img_url_api, nome_final) if (DOWNLOAD_IMAGES and img_url_api) else only_filename_from_url(img_url_api))

    return {
        "marca": BRAND,
        "nome": nome_final,
        "subtitulo": "",
        "categoria": categoria_final or "outros",
        "quantidade": quantidade_final or "",
        "preco": preco,
        "beneficios": beneficios_final,
        "ingredientes": ingredientes_final,
        "tipo_pele": tipo_pele_final or "",
        "imagem": imagem_fn or "",
    }

# ========================= PIPELINE =========================
def scrape_oceane_products() -> List[Dict]:
    print("Coletando via VTEX Search API…")
    raw = api_iter_all()
    print(f"[INFO] Produtos recebidos: {len(raw)}")

    rows = []
    for i, pj in enumerate(raw, 1):
        item = parse_product(pj)
        if item:
            rows.append(item)
        if i % 20 == 0:
            print(f"[INFO] Processados {i}/{len(raw)}")
    return rows

def save_data(products_data: List[Dict], json_path="oceane_products.json", csv_path="oceane_products.csv"):
    if not products_data:
        print("Nenhum dado para salvar.")
        return
    cols = ["marca","nome","subtitulo","categoria","quantidade","preco","beneficios","ingredientes","tipo_pele","imagem"]
    clean = []
    for r in products_data:
        row = {c: (r.get(c) or "") for c in cols}
        clean.append(row)
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(clean, f, ensure_ascii=False, indent=2)
    pd.DataFrame(clean, columns=cols).to_csv(csv_path, index=False, encoding="utf-8")
    print(f"[OK] JSON: {os.path.abspath(json_path)}")
    print(f"[OK] CSV : {os.path.abspath(csv_path)}")
    print(f"[OK] Itens salvos: {len(clean)}")
    print(f"[OK] Imagens em: {IMAGES_DIR.resolve()}")

# ========================= RUN =========================
if __name__ == "__main__":
    data = scrape_oceane_products()
    # ordena pra facilitar revisão
    data = sorted(data, key=lambda x: (x.get("categoria",""), x.get("nome","")))
    save_data(data)


Coletando via VTEX Search API…
[API] +50 (total 50)
[API] +50 (total 100)
[API] +50 (total 150)
[API] +50 (total 200)
[API] +50 (total 250)
[API] +24 (total 274)
[INFO] Produtos recebidos: 274


/tmp/ipykernel_16200/1755377021.py:303: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for tag in soup.find_all(text=re.compile(r"(?i)composi[cç][aã]o|ingredientes")):


[INFO] Processados 20/274
[INFO] Processados 40/274
[INFO] Processados 60/274
[INFO] Processados 80/274
[INFO] Processados 100/274
[INFO] Processados 120/274
[INFO] Processados 140/274
[INFO] Processados 160/274
[INFO] Processados 180/274
[INFO] Processados 200/274
[INFO] Processados 220/274
[INFO] Processados 240/274
[INFO] Processados 260/274
[OK] JSON: /home/usuario/Área de trabalho/Dados/Oceane/oceane_products.json
[OK] CSV : /home/usuario/Área de trabalho/Dados/Oceane/oceane_products.csv
[OK] Itens salvos: 107
[OK] Imagens em: /home/usuario/Área de trabalho/Dados/Oceane/images
